In [9]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [10]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [11]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [12]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110630

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,188869.3318
2018-02-28,150087.6827
2018-03-31,313652.4587
2018-04-30,234184.9218
2018-05-31,227723.1473
2018-06-30,245117.8843
2018-07-31,209520.1730
2018-08-31,210180.5684
2018-09-30,254540.9785


In [13]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    188869.3318
 2018-02-28    150087.6827
 2018-03-31    313652.4587
 2018-04-30    234184.9218
 2018-05-31    227723.1473
 2018-06-30    245117.8843
 2018-07-31    209520.1730
 2018-08-31    210180.5684
 2018-09-30    254540.9785
 2018-10-31    226399.2114
 2018-11-30    208042.1473
 2018-12-31    208632.4683
 2019-01-31    203305.4439
 2019-02-28    176692.8384
 2019-03-31    312059.7727
 2019-04-30    292091.6429
 2019-05-31    259072.3798
 2019-06-30    253572.8188
 2019-07-31    234290.1410
 2019-08-31    245949.9763
 2019-09-30    257164.6243
 2019-10-31    234505.3070
 2019-11-30    242069.4600
 2019-12-31    255480.9624
 2020-01-31    200521.7774
 2020-02-29    252254.1962
 2020-03-31    234639.7711
 2020-04-30    264439.0647
 2020-05-31    199138.0778
 2020-06-30    188309.0088
 2020-07-31    189297.2797
 2020-08-31    194295.3791
 2020-09-30    173751.8990
 2020-10-31    270165.7733
 2020-11-30    237079.0589
 2020-12-31    205265.9297

# ARIMA

In [14]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [15]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -5.307695
p-value : 0.000005
Critical Values :
	1%: -3.600983
	5%: -2.935135
	10%: -2.605963


In [16]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1008.106, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1009.873, Time=0.02 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1011.038, Time=0.03 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1012.712, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=1017.171, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=1018.170, Time=0.09 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1009.029, Time=0.04 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1012.527, Time=0.04 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1013.490, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1014.761, Time=0.08 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=1020.124, Time=0.18 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1010.946, Time=0.07 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1014.190, Time=0.08 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1015.346, Time=0.17 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.43 sec
 ARIMA(3,0,0)(0

ARIMA(order=(0, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [17]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 0)   Log Likelihood                -502.053
Method:                           css   S.D. of innovations          37597.911
Date:                Sat, 11 Sep 2021   AIC                           1008.106
Time:                        04:15:16   BIC                           1011.581
Sample:                             0   HQIC                          1009.380
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        2.25e+05   5801.484     38.781      0.000    2.14e+05    2.36e+05


C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [18]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([224989.52810952, 224989.52810952]), array([37597.91109688, 37597.91109688]), array([[151298.97646571, 298680.07975334],
       [151298.97646571, 298680.07975334]]))
